In [31]:
import pymysql
import pandas as pd
from datetime import datetime
import numpy as np

In [32]:
def judge_new_student(create_time):
    '''判断学生是否大班课加入的新生'''
    create_time_str = create_time.strftime('%Y-%m-%d %H:%M:%S')
    if create_time_str >= '2020-07-17 00:00:00':
        return 1
    return 0

def has_recharge(recharge_time):
    '''判断学生是否上了大班课充值'''
    if pd.isna(recharge_time):
        return 0
#     recharge_time_str = recharge_time.strftime('%Y-%m-%d %H:%M:%S')
    if recharge_time >= '2020-07-17 00:00:00':
        return 1
    return 0

def join_class_time(join_time):
    join_time_str = join_time.strftime('%Y-%m-%d %H:%M:%S')
    if join_time_str <= '2020-07-18 16:55:00':
        return 1
    elif join_time_str <= '2020-07-19 16:55:00':
        return 2
    elif join_time_str <= '2020-07-24 16:55:00':
        return 3
    elif join_time_str <= '2020-07-25 16:55:00':
        return 4
    elif join_time_str <= '2020-07-26 08:55:00':
        return 5
    return 6

def join_classroom(join_time):
    join_time_str = join_time.strftime('%Y-%m-%d %H:%M:%S')
    if join_time_str <= '2020-07-18 08:55:00':
        return 1
    elif join_time_str <= '2020-07-18 16:55:00':
        return 2
    elif join_time_str <= '2020-07-19 08:55:00':
        return 1
    elif join_time_str <= '2020-07-19 16:55:00':
        return 2
    elif join_time_str <= '2020-07-24 08:55:00':
        return 1
    elif join_time_str <= '2020-07-24 16:55:00':
        return 2
    elif join_time_str <= '2020-07-25 08:55:00':
        return 1
    elif join_time_str <= '2020-07-25 16:55:00':
        return 2
    elif join_time_str <= '2020-07-26 08:55:00':
        return 1
    elif join_time_str <= '2020-07-26 16:55:00':
        return 2
    return 3

def student_attend_or_not(student_id, student_ids):
    if student_id in student_ids:
        return 1
    return 0

def student_id_str(student_user_id):
    return 'sid_' + str(student_user_id)

def parent_id_str(parent_user_id):
    return 'pid_' + str(parent_user_id)

In [33]:
def data_out_sql(sql):
    #conn = pymysql.connect(host=DB['host'], port=DB['port'], user=DB['user'], passwd=DB['password'], db=DB['dbname'],charset='utf8')
#     conn = pymysql.connect(user='lingoace', password='DB4lingo.com', database='data_analysis')
    # 使用异步连接mysql
    conn = pymysql.connect(user='lingoace', password='DB4lingo.com', database='lingoacedb')

    # 创建游标
    cursor = conn.cursor()
    # 执行sql语句
    try:
        cursor.execute(sql)
        # 调出数据
        data = cursor.fetchall()
    

In [34]:
#常用数据存放目录
catelog='./data/'
#临时数据存放目录
tempData='./temp/'
#数据库数据数据定义
Database='lingoacedb'

def data_to_csv(df, name):
    
    catelog='./data/'

    name_suffix = datetime.now().strftime('%Y%m%d')
    
    df.to_csv('%s%s%s.csv' % (catelog, name, name_suffix), encoding='utf_8_sig')

In [35]:
#常用数据存放目录
catelog='./data/'
#临时数据存放目录
tempData='./temp/'
#数据库数据数据定义
Database='lingoacedb'

def data_to_csv(df, name):
    
    catelog='./data/'

    name_suffix = datetime.now().strftime('%Y%m%d')
    
    df.to_csv('%s%s%s.csv' % (catelog, name, name_suffix), encoding='utf_8_sig')

In [36]:
big_student_sql = '''select 
        distinct 
        usi.id,usi.real_name as student_name, 
        usi.parent_user_id, upi.username, 
        CONVERT_TZ(usi.create_time,'+00:00','+08:00') as create_time,
        (select max(CONVERT_TZ(fro.update_time,'+00:00','+08:00')) from finance_recharge_order fro where fro.parent_user_id=upi.id) as recharge_time
    from schedule_virtualclass_member svm 
    left join user_student_info usi on usi.id=svm.student_user_id
    left join user_parent_info upi on upi.id=usi.parent_user_id
    where svm.virtual_class_id in(
6540615750751332,
3806623631941512,
4491716293023605,
2398789452170581,
7775398486494051,
7683545376241951,
3889194356179761,
8435164883562844,
1585092767233948,
4468910679022166
    )'''

In [37]:
big_student = data_out_sql(big_student_sql)
big_student.head()

,id,student_name,parent_user_id,username,create_time,recharge_time
0,4617788762859180,None,7270530213994187,+8613051201801,2020-05-19 22:59:59,None
1,4488675914812301,test0007,3294449481294229,+8617000000007,2020-07-15 10:40:50,None
2,6902342807417957,7007,3294449481294229,+8617000000007,2020-05-20 14:57:48,None
3,8130603107956517,None,7314254430192790,8618518679681,2020-05-18 22:48:58,None
4,8901421693869038,小熊,5894466209541920,sunhan,2020-04-19 18:58:07,2020-07-18 20:55:00


判断学生是否是新学生

In [38]:
big_student['new_student'] = big_student.apply(lambda x: judge_new_student(x['create_time']), axis=1)
big_student['has_recharge'] = big_student.apply(lambda x: has_recharge(x['recharge_time']), axis=1)
big_student.head()

,id,student_name,parent_user_id,username,create_time,recharge_time,new_student,has_recharge
0,4617788762859180,None,7270530213994187,+8613051201801,2020-05-19 22:59:59,None,0,0
1,4488675914812301,test0007,3294449481294229,+8617000000007,2020-07-15 10:40:50,None,0,0
2,6902342807417957,7007,3294449481294229,+8617000000007,2020-05-20 14:57:48,None,0,0
3,8130603107956517,None,7314254430192790,8618518679681,2020-05-18 22:48:58,None,0,0
4,8901421693869038,小熊,5894466209541920,sunhan,2020-04-19 18:58:07,2020-07-18 20:55:00,0,1


In [39]:
student_join_time_sql = '''select student_user_id, min(CONVERT_TZ(create_time, '+00:00', '+08:00')) as join_time from schedule_virtualclass_member where virtual_class_id in(
    6540615750751332,
3806623631941512,
4491716293023605,
2398789452170581,
7775398486494051,
7683545376241951,
3889194356179761,
8435164883562844,
1585092767233948,
4468910679022166) group by student_user_id'''

In [40]:
student_join_time = data_out_sql(student_join_time_sql)
big_student = pd.merge(big_student, 
                       student_join_time, 
                       how='left', 
                       left_on='id', 
                       right_on='student_user_id')
del big_student['student_user_id']
big_student.head()

,id,student_name,parent_user_id,username,create_time,recharge_time,new_student,has_recharge,join_time
0,4617788762859180,None,7270530213994187,+8613051201801,2020-05-19 22:59:59,None,0,0,2020-07-15 11:32:06
1,4488675914812301,test0007,3294449481294229,+8617000000007,2020-07-15 10:40:50,None,0,0,2020-07-15 11:33:05
2,6902342807417957,7007,3294449481294229,+8617000000007,2020-05-20 14:57:48,None,0,0,2020-07-15 11:33:05
3,8130603107956517,None,7314254430192790,8618518679681,2020-05-18 22:48:58,None,0,0,2020-07-15 11:34:41
4,8901421693869038,小熊,5894466209541920,sunhan,2020-04-19 18:58:07,2020-07-18 20:55:00,0,1,2020-07-15 11:39:34


In [41]:
big_student['join_class_time'] = big_student.apply(lambda x: join_class_time(x['join_time']), axis=1)

第一课

In [42]:
first_class_attend_student_sql = '''select student_user_id from schedule_virtualclass_member where virtual_class_id in (    
    6540615750751332,
    3806623631941512) and enter_time is not null'''

first_class_attend_student = data_out_sql(first_class_attend_student_sql)
first_attend_student_ids = first_class_attend_student['student_user_id'].values.tolist()


In [43]:
big_student['first_class'] = big_student.apply(lambda x: student_attend_or_not(x['id'], first_attend_student_ids), axis=1)
big_student.head()

,id,student_name,parent_user_id,username,create_time,recharge_time,new_student,has_recharge,join_time,join_class_time,first_class
0,4617788762859180,None,7270530213994187,+8613051201801,2020-05-19 22:59:59,None,0,0,2020-07-15 11:32:06,1,0
1,4488675914812301,test0007,3294449481294229,+8617000000007,2020-07-15 10:40:50,None,0,0,2020-07-15 11:33:05,1,0
2,6902342807417957,7007,3294449481294229,+8617000000007,2020-05-20 14:57:48,None,0,0,2020-07-15 11:33:05,1,0
3,8130603107956517,None,7314254430192790,8618518679681,2020-05-18 22:48:58,None,0,0,2020-07-15 11:34:41,1,0
4,8901421693869038,小熊,5894466209541920,sunhan,2020-04-19 18:58:07,2020-07-18 20:55:00,0,1,2020-07-15 11:39:34,1,1


In [44]:
second_class_attend_student_sql = '''select student_user_id from schedule_virtualclass_member where virtual_class_id in (    
    4491716293023605,
    2398789452170581) and enter_time is not null'''

second_class_attend_student = data_out_sql(second_class_attend_student_sql)
second_attend_student_ids = second_class_attend_student['student_user_id'].values.tolist()

In [45]:
big_student['second_class'] = big_student.apply(lambda x: student_attend_or_not(x['id'], second_attend_student_ids), axis=1)
big_student.head()

,id,student_name,parent_user_id,username,create_time,recharge_time,new_student,has_recharge,join_time,join_class_time,first_class,second_class
0,4617788762859180,None,7270530213994187,+8613051201801,2020-05-19 22:59:59,None,0,0,2020-07-15 11:32:06,1,0,0
1,4488675914812301,test0007,3294449481294229,+8617000000007,2020-07-15 10:40:50,None,0,0,2020-07-15 11:33:05,1,0,0
2,6902342807417957,7007,3294449481294229,+8617000000007,2020-05-20 14:57:48,None,0,0,2020-07-15 11:33:05,1,0,0
3,8130603107956517,None,7314254430192790,8618518679681,2020-05-18 22:48:58,None,0,0,2020-07-15 11:34:41,1,0,0
4,8901421693869038,小熊,5894466209541920,sunhan,2020-04-19 18:58:07,2020-07-18 20:55:00,0,1,2020-07-15 11:39:34,1,1,0


In [46]:
third_class_attend_student_sql = '''select student_user_id from schedule_virtualclass_member where virtual_class_id in (    
    7775398486494051,
    7683545376241951) and enter_time is not null'''

third_class_attend_student = data_out_sql(third_class_attend_student_sql)
third_attend_student_ids = third_class_attend_student['student_user_id'].values.tolist()

In [47]:
big_student['third_class'] = big_student.apply(lambda x: student_attend_or_not(x['id'], third_attend_student_ids), axis=1)
big_student.head()

,id,student_name,parent_user_id,username,create_time,recharge_time,new_student,has_recharge,join_time,join_class_time,first_class,second_class,third_class
0,4617788762859180,None,7270530213994187,+8613051201801,2020-05-19 22:59:59,None,0,0,2020-07-15 11:32:06,1,0,0,0
1,4488675914812301,test0007,3294449481294229,+8617000000007,2020-07-15 10:40:50,None,0,0,2020-07-15 11:33:05,1,0,0,0
2,6902342807417957,7007,3294449481294229,+8617000000007,2020-05-20 14:57:48,None,0,0,2020-07-15 11:33:05,1,0,0,0
3,8130603107956517,None,7314254430192790,8618518679681,2020-05-18 22:48:58,None,0,0,2020-07-15 11:34:41,1,0,0,0
4,8901421693869038,小熊,5894466209541920,sunhan,2020-04-19 18:58:07,2020-07-18 20:55:00,0,1,2020-07-15 11:39:34,1,1,0,0


In [48]:
four_class_attend_student_sql = '''select student_user_id from schedule_virtualclass_member where virtual_class_id in (    
    3889194356179761,
    8435164883562844) and enter_time is not null'''

four_class_attend_student = data_out_sql(four_class_attend_student_sql)
four_attend_student_ids = four_class_attend_student['student_user_id'].values.tolist()

In [49]:
big_student['four_class'] = big_student.apply(lambda x: student_attend_or_not(x['id'], four_attend_student_ids), axis=1)
big_student.head()

,id,student_name,parent_user_id,username,create_time,recharge_time,new_student,has_recharge,join_time,join_class_time,first_class,second_class,third_class,four_class
0,4617788762859180,None,7270530213994187,+8613051201801,2020-05-19 22:59:59,None,0,0,2020-07-15 11:32:06,1,0,0,0,0
1,4488675914812301,test0007,3294449481294229,+8617000000007,2020-07-15 10:40:50,None,0,0,2020-07-15 11:33:05,1,0,0,0,0
2,6902342807417957,7007,3294449481294229,+8617000000007,2020-05-20 14:57:48,None,0,0,2020-07-15 11:33:05,1,0,0,0,0
3,8130603107956517,None,7314254430192790,8618518679681,2020-05-18 22:48:58,None,0,0,2020-07-15 11:34:41,1,0,0,0,0
4,8901421693869038,小熊,5894466209541920,sunhan,2020-04-19 18:58:07,2020-07-18 20:55:00,0,1,2020-07-15 11:39:34,1,1,0,0,0


In [50]:
five_class_attend_student_sql = '''select student_user_id from schedule_virtualclass_member where virtual_class_id in (    
    1585092767233948,
    4468910679022166) and enter_time is not null'''

five_class_attend_student = data_out_sql(five_class_attend_student_sql)
five_attend_student_ids = five_class_attend_student['student_user_id'].values.tolist()

In [51]:
big_student['five_class'] = big_student.apply(lambda x: student_attend_or_not(x['id'], five_attend_student_ids), axis=1)
big_student.head()

,id,student_name,parent_user_id,username,create_time,recharge_time,new_student,has_recharge,join_time,join_class_time,first_class,second_class,third_class,four_class,five_class
0,4617788762859180,None,7270530213994187,+8613051201801,2020-05-19 22:59:59,None,0,0,2020-07-15 11:32:06,1,0,0,0,0,0
1,4488675914812301,test0007,3294449481294229,+8617000000007,2020-07-15 10:40:50,None,0,0,2020-07-15 11:33:05,1,0,0,0,0,0
2,6902342807417957,7007,3294449481294229,+8617000000007,2020-05-20 14:57:48,None,0,0,2020-07-15 11:33:05,1,0,0,0,0,0
3,8130603107956517,None,7314254430192790,8618518679681,2020-05-18 22:48:58,None,0,0,2020-07-15 11:34:41,1,0,0,0,0,0
4,8901421693869038,小熊,5894466209541920,sunhan,2020-04-19 18:58:07,2020-07-18 20:55:00,0,1,2020-07-15 11:39:34,1,1,0,0,0,0


In [52]:
big_student['id'] = big_student.apply(lambda x: student_id_str(x['id']), axis=1)
big_student['parent_user_id'] = big_student.apply(lambda x: parent_id_str(x['parent_user_id']), axis=1)


In [53]:
data_to_csv(big_student, 'big_class_student')